In [1]:
import pandas as pd

In [2]:
import os

In [4]:
df = pd.read_csv(os.path.join("./spreadsheets/", "reflacx_clinical_eye.csv"))

In [6]:
df.head(5)

,Unnamed: 0,id,dicom_id,subject_id,stay_id,study_id,split,eye_tracking_data_discarded,image_path,bbox_path,...,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,chiefcomplaint,age
0,0,P102R108387,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002.0,55032240,train,False,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\mai...,...,98.1,90.0,18.0,99.0,184.0,75.0,13,3.0,Dyspnea,69
1,1,P102R379837,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002.0,55032240,test,False,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\mai...,...,98.1,90.0,18.0,99.0,184.0,75.0,13,3.0,Dyspnea,69
2,2,P102R558314,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002.0,55032240,test,False,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\mai...,...,98.1,90.0,18.0,99.0,184.0,75.0,13,3.0,Dyspnea,69
3,3,P102R765317,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002.0,55032240,train,False,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\mai...,...,98.1,90.0,18.0,99.0,184.0,75.0,13,3.0,Dyspnea,69
4,4,P102R915878,34cedb74-d0996b40-6d218312-a9174bea-d48dc033,18111516,32067002.0,55032240,train,False,{XAMI_MIMIC_PATH}\patient_18111516\CXR-JPG\s55...,{XAMI_MIMIC_PATH}\patient_18111516\REFLACX\mai...,...,98.1,90.0,18.0,99.0,184.0,75.0,13,3.0,Dyspnea,69


In [13]:
# df.transpose().values.tolist()[1]

In [15]:
import torch

a = torch.arange(15)

In [17]:
import torch.nn as nn

In [30]:
loss = nn.CrossEntropyLoss()

tensor([1, 1, 1, 1, 1])

In [64]:
batch_size =5
r = torch.randn((batch_size, 10))

In [68]:
loss(r, torch.ones_like(torch.arange(batch_size)))

tensor(3.0273)

In [69]:

loss(r, torch.arange(batch_size))

tensor(2.5424)

In [37]:
loss(torch.zeros((batch_size, 10)), torch.tensor([0]))

tensor(2.3026)

In [ ]:
loss(torch.zeros((batch_size, 10)), torch.arange([0]))

In [40]:
loss(torch.tensor([[1, 0, 0]]).float(), torch.tensor([0]))

tensor(0.5514)

In [47]:
loss(torch.tensor([[3, 0, 0]]).float(), torch.tensor([0]))

tensor(0.0949)

In [38]:
torch.zeros((batch_size, 10))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [34]:
torch.arange(batch_size)

tensor([0])

In [28]:
torch.arange(batch_size)

tensor([0])

In [21]:
torch.arange(batch_size)

tensor([0, 1, 2, 3, 4])

In [ ]:

class CLIPLoss(torch.nn.Module):
  """
  Loss function for multimodal contrastive learning based off of the CLIP paper.
  
  Embeddings are taken, L2 normalized and dot product between modalities is calculated to generate a cosine
  similarity between all combinations of subjects in a cross-modal fashion. Tempered by temperature.
  Loss is calculated attempting to match each subject's embeddings between the modalities i.e. the diagonal. 
  """
  def __init__(self, 
               temperature: float,
               lambda_0: float = 0.5) -> None:
    super(CLIPLoss, self).__init__()

    self.temperature = temperature
    self.cross_entropy = nn.CrossEntropyLoss(reduction='mean')

    if lambda_0 > 1 or lambda_0 < 0:
      raise ValueError('lambda_0 must be a float between 0 and 1.')
    self.lambda_0 = lambda_0
    self.lambda_1 = 1-lambda_0

  def forward(self, out0: torch.Tensor, out1: torch.Tensor, indices: List[int] = None) -> Tuple:
    # normalize the embedding onto the unit hypersphere
    out0 = nn.functional.normalize(out0, dim=1)
    out1 = nn.functional.normalize(out1, dim=1)

    #logits = torch.matmul(out0, out1.T) * torch.exp(torch.tensor(self.temperature))
    logits = torch.matmul(out0, out1.T) / self.temperature # the cosine similarity function is (A \dot B) / ||A|| ||B||, since ||A|| = ||B|| = 1, we only need to calculate the molecular term.
    labels = torch.arange(len(out0), device=out0.device) # a list of [0, 1, 2, 3....] as labels? why?
    
    loss_0 = self.lambda_0 * self.cross_entropy(logits, labels)
    loss_1 = self.lambda_1 * self.cross_entropy(logits.T, labels)
    loss = loss_0 + loss_1
  
    return loss, logits, labels